In [ ]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By

import random
from time import sleep

USERNAME = ''
PASSWORD = '' 

USERNAME_nathan = ''
PASSWORD_nathan = ''

n_one_room = 2
n_dubbel_room = 0
n_two_room = 1
n_three_room = 2

max_price_one_room = 5500

min_surface_one_room = 21
min_surface_two_room = 45
min_surface_three_room = 50
date_arrival = "2021-09-01"

# SIMPLE ROUTINE LOGIN -> VACANT OBJECTS -> FILTER -> LIST VACANT OBJECTS


class browsing_bot():
    def __init__(self, username, password):
        self.BASE_URL = 'https://www.sgs.se/Mina-sidor/Login?return=https://www.sgs.se/mina-sidor/mina-%C3%A4renden?sc_lang=en'
        self.VACANT_OBJECT_URL = 'https://www.sgs.se/mina-sidor/lediga-objekt?sc_lang=en'
        
        self.username = username
        self.password = password
        
        self.chromedriver = 'chromedriver.exe'
        self.browser = webdriver.Chrome(self.chromedriver)
        self.browser.set_window_size(900,1080)
    
    def login(self):
        self.browser.get(self.BASE_URL)
        username = self.browser.find_element_by_name("User")
        username.send_keys(self.username)

        password = self.browser.find_element_by_name("Password")
        password.send_keys(self.password)

        password.send_keys(Keys.RETURN)
        sleep(3)
        self.browser.get(self.VACANT_OBJECT_URL)

    # FILTER
    def my_filter(self):
        sleep(5)
        # switch to filter frame
        filter_frame = self.browser.find_element_by_id("momentum")
        self.browser.switch_to_frame(filter_frame)

        # display toggle btn
        toggle_search_area = self.browser.find_element_by_id("toggle-seekarea")
        toggle_search_area.send_keys(Keys.ENTER)

        sleep(2)

    def close(self):
        sleep(1000)
        self.browser.close()

In [ ]:
def send_mail(filtered_object):
    message = """\
    """
    body = '\n\nThe number of accomodation has changed.\n'
    body+= "There are {} accomodation in your filter\n".format(len(filtered_object))

    for valid_accomodation in filtered_object:
        body += "Name {} Surface {} Price {} Room {}\n".format(valid_accomodation[4].text, valid_accomodation[2], valid_accomodation[1], valid_accomodation[3])               
        
    message = message + body
    do_mail("[SGS UPDATE INFO]", message)
    
    

In [ ]:
def send_mail_applied(name, date_extract, price_extract, surface_extract, number_of_room_extract):
    message = """\
    """
    body = '\n\nWe applied for an accomodation.\n'
    body += "Name {} Surface {} Price {} Room {}\n".format(name, surface_extract, price_extract, number_of_room_extract)               
        
    message = message + body
    do_mail("[SGS APPLY UPDATED]",message)

In [ ]:
def do_mail(subject, message_str):
    import smtplib, ssl
    from email.mime.text import MIMEText
    from email.mime.multipart import MIMEMultipart
    import unicodedata

    
    password = "qf5vw89fz14ap04"
    port = 587  # For starttls
    smtp_server = "smtp.gmail.com"
    sender_email = "hugo.mathh@gmail.com"
    receiver_email = "hugo.mathh@gmail.com"
    receiver_email_2 = "nathan.haudot@outlook.fr"

    message = MIMEMultipart("alternative")
    message["Subject"] = subject
    message["From"] = sender_email
    message["To"] = receiver_email
    
    part = MIMEText(message_str, "plain", 'utf-8')
    message.attach(part)

    context = ssl.create_default_context()
    #print("Msg to send : {}".format(message))

    
    s = smtplib.SMTP(smtp_server, 587, timeout=10)
    s.set_debuglevel(1)
    try:
        s.starttls()
        s.login(sender_email, password)
        s.sendmail(
            sender_email, receiver_email, message.as_string()
        )
        s.sendmail(
            sender_email, receiver_email_2, message.as_string()
        )
    finally:
        s.quit()
        

In [ ]:
def filter_object(surface_, price, date, number_of_room, url):
    global date_arrival, n_one_room, n_dubbel_room, n_two_room , n_three_room,max_price_one_room, min_surface_one_room, min_surface_two_room, min_surface_three_room 
    
    date = date.text.split(" ")[2]
    surface = surface_.text.split(" ")[0]
    number_of_room = int(number_of_room.text.split(" ")[0])
    price = price.text.split(" ")
    price = int(price[1]+price[2])
    surface = int(float(surface))
    
    print("Begining date {} surface {} number room {} price {}".format(date, surface, number_of_room, price))
   
    if (int(date.split("-")[1]) == int(date_arrival.split("-")[1])): # date != date_arrival
        if (number_of_room == 1):
            if ( surface >= min_surface_one_room):
                if (price < max_price_one_room):
                    return True, date, price, surface, number_of_room, url
        elif (number_of_room == 2):
            if (surface >= min_surface_two_room):
                return True, date, price, surface, number_of_room, url
        elif (number_of_room == 3):
            if (surface >= min_surface_three_room):
                return True, date, price, int(surface), number_of_room, url
    return False, date, price, surface, number_of_room, url

In [28]:
class Routine:
    def __init__(self):
        self.browser = browsing_bot(USERNAME_nathan, PASSWORD_nathan)
        self.number_of_accomodation = 0
        self.ancient_accomodation_filtered = []
    def login(self):
        self.browser.login()

    def filter_search_sgs(self):
        filter_frame = self.browser.browser.find_element_by_id("momentum")
        self.browser.browser.switch_to_frame(filter_frame)
        sleep(0.5)
        toggle_search_area = self.browser.browser.find_element_by_id("toggle-seekarea")
        toggle_search_area.send_keys(Keys.ENTER)
        sleep(0.5)
        # FILTER CENTER
        center_btn = self.browser.browser.find_element_by_xpath("/html/body/div[2]/form/div[3]/div[4]/div[1]/div[2]/div[2]/div/div[1]/div[2]/div[1]/div[2]/ul/li/ul/li[1]/a/ins[1]").click()
        # FILTER NORR
        norr_expand_btn = self.browser.browser.find_element_by_xpath("/html/body/div[2]/form/div[3]/div[4]/div[1]/div[2]/div[2]/div/div[1]/div[2]/div[1]/div[2]/ul/li/ul/li[4]/ins").click()
        lindholmsallen = self.browser.browser.find_element_by_xpath("/html/body/div[2]/form/div[3]/div[4]/div[1]/div[2]/div[2]/div/div[1]/div[2]/div[1]/div[2]/ul/li/ul/li[4]/ul/li[3]/a/ins[1]").click()
        plejaden = self.browser.browser.find_element_by_xpath("/html/body/div[2]/form/div[3]/div[4]/div[1]/div[2]/div[2]/div/div[1]/div[2]/div[1]/div[2]/ul/li/ul/li[4]/ul/li[4]/a/ins[1]").click()
        centrala_hisingen = self.browser.browser.find_element_by_xpath("/html/body/div[2]/form/div[3]/div[4]/div[1]/div[2]/div[2]/div/div[1]/div[2]/div[1]/div[2]/ul/li/ul/li[4]/ul/li[7]/a/ins[1]").click()
            # FILTER RUM
        alla = self.browser.browser.find_element_by_xpath("/html/body/div[2]/form/div[3]/div[4]/div[1]/div[2]/div[2]/div/div[1]/div[1]/div[4]/span/div/button").click()
        # 1 rum 
        one_rum = self.browser.browser.find_element_by_xpath("/html/body/div[2]/form/div[3]/div[4]/div[1]/div[2]/div[2]/div/div[1]/div[1]/div[4]/span/div/ul/li[2]/a/label/input").click()
        # 2 rum
        two_rum = self.browser.browser.find_element_by_xpath("/html/body/div[2]/form/div[3]/div[4]/div[1]/div[2]/div[2]/div/div[1]/div[1]/div[4]/span/div/ul/li[3]/a/label").click()
        # 3 rum
        three_rum = self.browser.browser.find_element_by_xpath("/html/body/div[2]/form/div[3]/div[4]/div[1]/div[2]/div[2]/div/div[1]/div[1]/div[4]/span/div/ul/li[4]/a/label").click()
        # dubbelrum
        dubbelrum = self.browser.browser.find_element_by_xpath("/html/body/div[2]/form/div[3]/div[4]/div[1]/div[2]/div[2]/div/div[1]/div[1]/div[4]/span/div/ul/li[9]/a/label").click()

    def search_and_list_sgs(self):
        # SEARCH
        self.browser.browser.find_element_by_xpath("/html/body/div[2]/form/div[3]/div[4]/div[1]/div[2]/div[2]/div/div[1]/div[4]/div/div/a[3]").click()
        # vew all vacant objects in one page 
        sleep(0.5)
        view_all = self.browser.browser.find_element_by_xpath("/html/body/div[2]/form/div[3]/div[4]/div[1]/div[2]/div[4]/div[2]/div[2]/div/select").click()
        sleep(0.5)
        put_all_vacant = self.browser.browser.find_element_by_xpath("/html/body/div[2]/form/div[3]/div[4]/div[1]/div[2]/div[4]/div[2]/div[2]/div/select/option[5]").click()
    
    def do_filter_object_sgs(self):
        # GET LIST OF VACANT OBJECT IN ONE PAGE
        list_vacant_object_one = self.browser.browser.find_element_by_xpath("/html/body/div[2]/form/div[3]/div[4]/div[1]/div[2]/div[4]/div[3]/div[1]/table/tbody")
        list_vacant = list_vacant_object_one.find_elements(By.TAG_NAME, "tr")
        print("Currently {} vacant objects in our filters".format(len(list_vacant)))

                # Print filtered object
        vacant_object = []
        self.filtered_object = []
        random.shuffle(list_vacant)
        # list of object
        sleep(1)
        for i in range(1, len(list_vacant)+1):
            date = "//*[@id='search-container']/table/tbody/tr["+str(i)+"]/td[3]/div[6]"
            price = "//*[@id='search-container']/table/tbody/tr["+str(i)+"]/td[3]/div[5]"
            surface = "//*[@id='search-container']/table/tbody/tr["+str(i)+"]/td[3]/div[4]"
            number_of_room = "//*[@id='search-container']/table/tbody/tr["+str(i)+"]/td[3]/div[3]"
            url_apply = "//*[@id='search-container']/table/tbody/tr["+str(i)+"]/td[3]/a[1]"
            
            begining_date_elemm = self.browser.browser.find_element_by_xpath(date)
            price_elem = self.browser.browser.find_element_by_xpath(price)
            surface_elem = self.browser.browser.find_element_by_xpath(surface)
            number_of_room = self.browser.browser.find_element_by_xpath(number_of_room)
            url_apply = self.browser.browser.find_element_by_xpath(url_apply)
            
            vacant_object.append([begining_date_elemm, price_elem, surface_elem, number_of_room, url_apply])

        for date, price, surface, number_of_room, url_apply in vacant_object:
            print("Available at {} Surface {} Number of room {} Price {} Url {}".format(date.text, surface.text, number_of_room.text, price.text, url_apply.text))
            filtered, date_extract, price_extract, surface_extract, number_of_room_extract, name = filter_object(surface, price, begining_date_elemm, number_of_room, url_apply)
            if (filtered):
                self.filtered_object.append([date_extract, price_extract, surface_extract, number_of_room_extract, name])
                msg = "Filtered : name {} date {} surface {} number room {} price {}\n".format(name.text, date.text, surface.text, number_of_room.text, price.text)
                print(msg)
        print("{} Filtered accomodation \n".format(len(self.filtered_object)))
        self.ancient_accomodation_filtered = self.filtered_object

        if (self.number_of_accomodation != len(list_vacant)):
            # Applied randomly for new accomodation 
            #self.apply_randomly_for_new_accomodation()
            
            self.number_of_accomodation = len(list_vacant)
            send_mail(self.filtered_object)
            print("[!] Accomodation updated")
            return self.filtered_object

        else:
            print("[*] The number of accomodation didn't changed, we continue")
        
    def apply_randomly_for_new_accomodation(self):
        nb_applied_for_1_room = 0
        nb_applied_for_2_room = 0
        nb_applied_for_3_room = 0

        global n_one_room, n_two_room, n_three_room

        accomodation_to_apply_for = []
        print("[*] Applying randomly to object ..")
        for accomodation in self.filtered_object:
            # filter each accomodation to see if they are new to the list
            if accomodation in self.ancient_accomodation_filtered:
                # new accomodation
                print("[*] New accomodation detected ! ")
                if (accomodation[3] == 1 and nb_applied_for_1_room < n_one_room):
                    nb_applied_for_1_room+=1
                    self.apply_sgs(accomodation[0], accomodation[1], accomodation[2], accomodation[3], accomodation[4], accomodation[4].text)
                elif (accomodation[3] == 2 and nb_applied_for_2_room < n_two_room):
                    nb_applied_for_2_room+=1
                    self.apply_sgs(accomodation[0], accomodation[1], accomodation[2], accomodation[3], accomodation[4], accomodation[4].text)
                elif (accomodation[3] == 3 and nb_applied_for_3_room < n_three_room):
                    nb_applied_for_3_room+=1
                    self.apply_sgs(accomodation[0], accomodation[1], accomodation[2], accomodation[3], accomodation[4], accomodation[4].text)
                else:
                    print("[*] There are no interests available !")
                    break
            else:
                print("[!] This accomodation is already here ! ")



        print("[*] Done")

    def apply_sgs(self, date_extract, price_extract, surface_extract, number_of_room_extract, url, name):
        print("[*] Applying for : {} ..".format(name))
        url.send_keys(Keys.ENTER)
        sleep(2)
        apply_url = self.browser.browser.find_element_by_xpath("/html/body/div[2]/form/div[3]/div[4]/div[1]/div[2]/div[5]/div[2]/div[1]/div[2]/div[2]/div/a[1]")
        apply_url.send_keys(Keys.ENTER)
        sleep(2)
        register_interest = self.browser.browser.find_element_by_xpath("/html/body/div[2]/form/div[3]/div[4]/div[1]/div[2]/div[8]/a")
        register_interest.send_keys(Keys.ENTER)
        print("[*] Done")
        print("[*] Sending mail ..")
        send_mail_applied(name, date_extract, price_extract, surface_extract, number_of_room_extract)
        print("[*] Done")
        sleep(2)
        print("[*] Back to object information ..")
        back_to_object = self.browser.browser.find_element_by_xpath("/html/body/div[2]/form/div[3]/div[4]/div[1]/div[2]/div[9]/div/a[1]")
        back_to_object.send_keys(Keys.ENTER)
        sleep(1)


In [ ]:
### BASIC ROUTINE

def basic_routine():
    routine_1 = Routine()
    routine_1.login()
    routine_1.filter_search_sgs()
    sleep(1)
    routine_1.search_and_list_sgs()
    sleep(2)
    print("[*] Starting search ..")
    routine_1.do_filter_object_sgs()
    print("[*] Done")
    # LOOP TO WATCH OUT FOR NEW ACCOMODATION
    while (True):
        sleep(50)
        print("[*] Reloading page ..")
        routine_1.browser.browser.get('https://www.sgs.se/mina-sidor/lediga-objekt?sc_lang=en')
        print("[*] Done")
        sleep(1)
        print("[*] Putting some criteria for the search ..")
        routine_1.filter_search_sgs()
        sleep(2)
        routine_1.search_and_list_sgs()
        sleep(2)
        print("[*] Done")
        print("[*] Starting search ..")
        routine_1.do_filter_object_sgs()
        print("[*] Done")

In [29]:
# TEST APPLIED TO ONE OBJECT
routine_1 = Routine()
routine_1.login()
routine_1.filter_search_sgs()
sleep(1)
routine_1.search_and_list_sgs()
sleep(2)
print("[*] Starting search ..")
filtered_object = routine_1.do_filter_object_sgs()
print("[*] Done")

C:\Users\hugom\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:11: DeprecationWarning: use driver.switch_to.frame instead
  # This is added back by InteractiveShellApp.init_path()


[*] Starting search ..
Currently 31 vacant objects in our filters
Available at Ledig fr.o.m. 2021-10-01 Surface 22.4 kvm Number of room 1 rum och kokskåp Price Hyra 3 533 Url Doktor Forselius Backe 54 Lgh 135
Begining date 2021-09-01 surface 22 number room 1 price 3533
Filtered : name Doktor Forselius Backe 54 Lgh 135 date Ledig fr.o.m. 2021-10-01 surface 22.4 kvm number room 1 rum och kokskåp price Hyra 3 533

Available at Ledig fr.o.m. 2021-10-01 Surface 22.1 kvm Number of room 1 rum och kokskåp Price Hyra 3 538 Url Doktor Forselius Backe 62 Lgh 15
Begining date 2021-09-01 surface 22 number room 1 price 3538
Filtered : name Doktor Forselius Backe 62 Lgh 15 date Ledig fr.o.m. 2021-10-01 surface 22.1 kvm number room 1 rum och kokskåp price Hyra 3 538

Available at Ledig fr.o.m. 2021-09-01 Surface 22.4 kvm Number of room 1 rum och kokskåp Price Hyra 3 540 Url Doktor Forselius Backe 56 Lgh 87
Begining date 2021-09-01 surface 22 number room 1 price 3540
Filtered : name Doktor Forselius Ba

send: 'ehlo pc-hugo.home\r\n'
reply: b'250-smtp.gmail.com at your service, [2a01:cb05:8193:4d00:88f9:5e95:8b9a:a7af]\r\n'
reply: b'250-SIZE 35882577\r\n'
reply: b'250-8BITMIME\r\n'
reply: b'250-STARTTLS\r\n'
reply: b'250-ENHANCEDSTATUSCODES\r\n'
reply: b'250-PIPELINING\r\n'
reply: b'250-CHUNKING\r\n'
reply: b'250 SMTPUTF8\r\n'
reply: retcode (250); Msg: b'smtp.gmail.com at your service, [2a01:cb05:8193:4d00:88f9:5e95:8b9a:a7af]\nSIZE 35882577\n8BITMIME\nSTARTTLS\nENHANCEDSTATUSCODES\nPIPELINING\nCHUNKING\nSMTPUTF8'
send: 'STARTTLS\r\n'
reply: b'220 2.0.0 Ready to start TLS\r\n'
reply: retcode (220); Msg: b'2.0.0 Ready to start TLS'
send: 'ehlo pc-hugo.home\r\n'
reply: b'250-smtp.gmail.com at your service, [2a01:cb05:8193:4d00:88f9:5e95:8b9a:a7af]\r\n'
reply: b'250-SIZE 35882577\r\n'
reply: b'250-8BITMIME\r\n'
reply: b'250-AUTH LOGIN PLAIN XOAUTH2 PLAIN-CLIENTTOKEN OAUTHBEARER XOAUTH\r\n'
reply: b'250-ENHANCEDSTATUSCODES\r\n'
reply: b'250-PIPELINING\r\n'
reply: b'250-CHUNKING\r\n'
reply

[*] Done
[*] Back to object information ..
[*] New accomodation detected ! 
[*] There are no interests available !
[*] Done


StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=91.0.4472.124)


In [ ]:
## TRY CATCH INFINITE LOOP WITHOUT CRASHING

try:
    basic_routine()
except:
    print("[*] Error, reloading the soft")
    basic_routine()
